## Importing the relevant packages

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.graphics.tsaplots as sgt
import statsmodels.tsa.stattools as sts
from statsmodels.tsa.arima.model import ARIMA
from scipy.stats.distributions import chi2 
from arch import arch_model
from math import sqrt
import seaborn as sns
sns.set()

## Importing the Data and Pre-processing 

In [3]:
url = './ignore/S3/L11/Index2018.csv'
raw_csv_data = pd.read_csv(url) 
df=raw_csv_data.copy()
df.date = pd.to_datetime(df.date, dayfirst = True)
df.set_index("date", inplace=True)
df=df.asfreq('b')
df=df.fillna(method='ffill')

In [4]:
df['market_value']=df.ftse

In [5]:
del df['spx']
del df['dax']
del df['ftse']
del df['nikkei']
size = int(len(df)*0.8)
df_train, df_test = df.iloc[:size], df.iloc[size:]

In [6]:
# import warnings
# warnings.filterwarnings("ignore")

## The LLR Test

In [7]:
def LLR_test(mod_1, mod_2, DF = 1):
    L1 = mod_1.fit(start_ar_lags = 11).llf
    L2 = mod_2.fit(start_ar_lags = 11).llf
    LR = (2*(L2-L1))    
    p = chi2.sf(LR, DF).round(3)
    return p

## Creating Returns

In [8]:
df_train['returns'] = df_train.market_value.pct_change(1)*100

/var/folders/2h/c7dkm6rj2w318lkvfrnrk2c00000gn/T/ipykernel_75153/3020228530.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train['returns'] = df_train.market_value.pct_change(1)*100


## The Simple GARCH Model

In [10]:
garch11 = arch_model(df_train['returns'][1:], mean='constant', vol="GARCH", p=1, q=1)
results_garch11 = garch11.fit(update_freq=5)
print(results_garch11.summary())

Iteration:      5,   Func. Count:     35,   Neg. LLF: 7010.712423623649
Iteration:     10,   Func. Count:     64,   Neg. LLF: 6970.058478409483
Optimization terminated successfully    (Exit mode 0)
            Current function value: 6970.058366189904
            Iterations: 13
            Function evaluations: 78
            Gradient evaluations: 13
                     Constant Mean - GARCH Model Results                      
Dep. Variable:                returns   R-squared:                       0.000
Mean Model:             Constant Mean   Adj. R-squared:                  0.000
Vol Model:                      GARCH   Log-Likelihood:               -6970.06
Distribution:                  Normal   AIC:                           13948.1
Method:            Maximum Likelihood   BIC:                           13974.2
                                        No. Observations:                 5020
Date:                Tue, Jan 10 2023   Df Residuals:                     5019
Time:          

## Higher-Lag GARCH Models

In [11]:
garch12 = arch_model(df_train['returns'][1:],
                     mean='constant', vol="GARCH", p=1, q=2)
results_garch12 = garch12.fit(update_freq=5)
print(results_garch12.summary())

Iteration:      5,   Func. Count:     40,   Neg. LLF: 6974.175070184585
Iteration:     10,   Func. Count:     71,   Neg. LLF: 6970.058392785186
Optimization terminated successfully    (Exit mode 0)
            Current function value: 6970.058366227395
            Iterations: 12
            Function evaluations: 83
            Gradient evaluations: 12
                     Constant Mean - GARCH Model Results                      
Dep. Variable:                returns   R-squared:                       0.000
Mean Model:             Constant Mean   Adj. R-squared:                  0.000
Vol Model:                      GARCH   Log-Likelihood:               -6970.06
Distribution:                  Normal   AIC:                           13950.1
Method:            Maximum Likelihood   BIC:                           13982.7
                                        No. Observations:                 5020
Date:                Tue, Jan 10 2023   Df Residuals:                     5019
Time:          

In [12]:
garch13 = arch_model(df_train['returns'][1:],
                     mean='constant', vol="GARCH", p=1, q=3)
results_garch13 = garch13.fit(update_freq=5)
print(results_garch13.summary())

Iteration:      5,   Func. Count:     47,   Neg. LLF: 7044.9128405154115
Iteration:     10,   Func. Count:     88,   Neg. LLF: 6973.181221444772
Iteration:     15,   Func. Count:    124,   Neg. LLF: 6970.0583673749625
Optimization terminated successfully    (Exit mode 0)
            Current function value: 6970.058366230056
            Iterations: 17
            Function evaluations: 137
            Gradient evaluations: 17
                     Constant Mean - GARCH Model Results                      
Dep. Variable:                returns   R-squared:                       0.000
Mean Model:             Constant Mean   Adj. R-squared:                  0.000
Vol Model:                      GARCH   Log-Likelihood:               -6970.06
Distribution:                  Normal   AIC:                           13952.1
Method:            Maximum Likelihood   BIC:                           13991.2
                                        No. Observations:                 5020
Date:              

In [13]:
garch21 = arch_model(df_train['returns'][1:],
                     mean='constant', vol="GARCH", p=2, q=1)
results_garch21 = garch21.fit(update_freq=5)
print(results_garch21.summary())

Iteration:      5,   Func. Count:     40,   Neg. LLF: 8793.711923408657
Iteration:     10,   Func. Count:     76,   Neg. LLF: 6967.731247473179
Optimization terminated successfully    (Exit mode 0)
            Current function value: 6967.731020076106
            Iterations: 12
            Function evaluations: 87
            Gradient evaluations: 12
                     Constant Mean - GARCH Model Results                      
Dep. Variable:                returns   R-squared:                       0.000
Mean Model:             Constant Mean   Adj. R-squared:                  0.000
Vol Model:                      GARCH   Log-Likelihood:               -6967.73
Distribution:                  Normal   AIC:                           13945.5
Method:            Maximum Likelihood   BIC:                           13978.1
                                        No. Observations:                 5020
Date:                Tue, Jan 10 2023   Df Residuals:                     5019
Time:          

    GARCH(1,1) is the best, no need for overly complicated models